In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
img=cv2.imread('image.jpeg',0)
min_val=np.min(img)
max_val=np.max(img)
stretched=(img-min_val)*255/(max_val-min_val)
plt.subplot(1,2,1)
plt.imshow(img,cmap='gray')
plt.subplot(1,2,2)
plt.imshow(stretched,cmap='gray')
plt.show()

In [ ]:
kernel=np.ones((5,5),np.float32)/25
filtered=cv2.filter2D(stretched,-1,kernel)
plt.subplot(1,2,1)
plt.imshow(stretched,cmap='gray')
plt.subplot(1,2,2)
plt.imshow(filtered,cmap='gray')
plt.show()


In [ ]:
plt.hist(filtered.ravel(),256,[0,256])
plt.show()



```
# ex-2
```



In [ ]:
import tensorflow_datasets as tfds

(train_ds, test_ds), ds_info = tfds.load(
    'horses_or_humans',
    split=['train', 'test'],
    as_supervised=True,  # Returns (image, label) tuples
    with_info=True
)

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
AUTOTUNE=tf.data.AUTOTUNE
image_size=(224,224)
batch_size=32
train_count=ds_info.splits['train'].num_examples
test_count=ds_info.splits['test'].num_examples
print(train_count)
print(test_count)

In [4]:
def preprocess(image,label):
    image=tf.image.resize(image,image_size)
    image=tf.cast(image,tf.float32)/255.0
    return image,label
train_ds=train_ds.map(preprocess, num_parallel_calls=AUTOTUNE)
test_ds=test_ds.map(preprocess, num_parallel_calls=AUTOTUNE)
train_ds=train_ds.cache().shuffle(1000).batch(batch_size).prefetch(AUTOTUNE)
test_ds=test_ds.batch(batch_size).prefetch(AUTOTUNE)


In [ ]:
images,labels=next(iter(train_ds))
for i in range(4):
    plt.subplot(2,2,i+1)
    plt.imshow(images[i].numpy())
    plt.title(int(labels[i].numpy()))
plt.show()


In [ ]:
base=tf.keras.applications.ResNet50(weights=None,include_top=False,input_shape=(224,224,3))
model=tf.keras.Sequential([
    base,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1,activation='sigmoid')
])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
history=model.fit(train_ds.take(20),validation_data=test_ds.take(10),epochs=3)
plt.plot(history.history['accuracy'],label='train')
plt.plot(history.history['val_accuracy'],label='val')
plt.legend()
plt.show()
loss,acc=model.evaluate(test_ds.take(10))
print(loss,acc)
